In [23]:
!conda install -c anaconda beautifulsoup4 
!conda install -c anaconda lxml
!conda install -c anaconda html5lib
!conda install -c anaconda requests

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2018.8.24          |           py35_1         139 KB  anaconda
    ca-certificates-2019.5.15  |                0         133 KB  anaconda
    beautifulsoup4-4.6.3       |           py35_0         140 KB  anaconda
    openssl-1.0.2s             |       h7b6447c_0         3.1 MB  anaconda
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0      anaconda
    ca-certificates: 2019.1.23-0          --> 2019.5.15-0       anaconda
    certifi:         2018.8.24-py35_1     --> 2018.8.24-py35_1  anaconda
    openssl:   

### Import required libraries

In [24]:
import requests
import pandas as pd
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')

### Insert Wikipedia table into variable

In [25]:
My_table = soup.find('table',{'class':'wikitable sortable'}) 

### Parse table and create dataframe according to the given conditions

In [26]:
table_rows = My_table.find_all('tr')

PostCode =[]
Borough = []
Neighbourhood = []

for tr in table_rows:
    td = tr.find_all('td')
    if (len(td) > 0 and td[1].text != 'Not assigned'): #check if it valid row and if borough!="Not Assigned"
        if td[0].text not in PostCode: #if postcode not present before then create new
            PostCode.append((td[0].text)) #append postcode in array
            a = td[1].find_all('a')
            if(len(a) > 0):
                Borough.append((((td[1].find('a')).get('title')).split(','))[0]) #append borough in array for hyperlink
            else:
                Borough.append((((td[1].text)).split(','))[0]) #append borough in array for non-hyperlink
            a = td[2].find_all('a')
            if(len(a) > 0):
                Neighbourhood.append((((td[2].find('a')).get('title')).split(','))[0]) #append neighbourhood in array for hyperlink
            else:   #append neighbourhood in array for non-hyperlink
                if td[2].text == 'Not assigned\n': 
                    Neighbourhood.append(((td[1].text).split(','))[0])#append borough if non assigned
                else:
                    Neighbourhood.append((td[2].text).replace('\n', '')) # append neighbourhood if non-hyperlink
        else: #if postcode is present in the array
            ind = PostCode.index(td[0].text)
            a = td[2].find_all('a')
            if(len(a) > 0):
                testneighbourhood = (((td[2].find('a')).get('title')).split(','))[0] #append neighbourhood in array for hyperlink
            else:   #append neighbourhood in array for non-hyperlink
                if td[2].text != 'Not assigned\n':
                    testneighbourhood = (td[2].text).replace('\n', '') # append neighbourhood if non-hyperlink
            if Borough[ind] == Neighbourhood[ind]:
                Neighbourhood[ind] = testneighbourhood
            else:
                Neighbourhood[ind] = Neighbourhood[ind] + ', ' + testneighbourhood           
        
df=pd.DataFrame()
df['PostCode'] = PostCode
df['Borough'] = Borough
df['Neighbourhood'] = Neighbourhood
df[0:10]

,PostCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront (Toronto), Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park (Toronto),Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


## DataFrame Shape

In [27]:
df.shape

(103, 3)